# 1. Word2Vec 모델 학습을 위한 전처리


## 1) Tag 데이터 로드

먼저 태그데이터를 읽어옵니다. <br>
pandas를 사용하여 읽어온 데이터를 데이터프레임 형태로 변환합니다. <br>
데이터는 MovieLens의 데이터셋을 사용하였습니다. <br>


In [2]:
import pandas as pd

tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)
tag_df.head(10)
# 유저별 영화별 태그 데이터입니다.


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195
5,14,318,justice,1442615192
6,14,480,Dinosaurs,1443148563
7,14,593,psychothriller,1444014286
8,14,1682,philosophy,1442615158
9,14,1682,surveillance,1442615167


## 2) 유저별 태그데이터 배열 생성
로드한 데이터를 잘라서 유저별 영화별 배열을 생성합니다. <br>

<br>

#### 예 : <br>
#### [유저1-영화1-태그1  ,  유저1-영화1-태그2  ,  유저1-영화1-태그3 , ...] <br>
가장 먼저 태그들이 정렬됩니다. 이때, 태그는 시간의 순서에 따라 정렬합니다. <br>
비슷한시간대에 비슷한 태그를 작성한다는 가정하에 정렬하였습니다. <br>

<br>

#### [유저1-영화2-태그1  ,  유저1-영화2-태그2  ,  유저1-영화2-태그3 , ...] <br>
영화가 바뀌면 새로운 어레이가 생성됩니다. <br>
유저1이 영화1번, 영화2번에 작성한 태그는 각각 다른 어레이에 들어갑니다. <br>

<br>

#### [유저2-영화1-태그1  ,  유저2-영화1-태그2  ,  유저2-영화1-태그3 , ...] <br>
유저 1이 작성한 태그가 끝나면 유저2의 태그가 시작됩니다. <br>
마찬가지로 같은영화에 작성한 태그는 같은 어레이에 들어가고 <br>
다른영화에 작성한 태그는 다른 어레이에 들어갑니다. <br>
유저2가 끝나면 유저n 까지 반복합니다. <br> 


In [3]:
from src.word2vec_mylib import tag_manager as tm


tag_array = tm.get_tag_by_user(tag_df, start_movie=110, start_user=14)

for i in range(0, 10):
    print(tag_array[i])
    
# 위의 방식대로 처리한 태그데이터가 담긴 배열입니다.

['medieval', 'epic']
['sci-fi', 'space action']
['justice', 'imdb top 250']
['dinosaurs']
['psychothriller']
['philosophy', 'surveillance']
['epic']
['pixar']
['dinosaurs']
['sci-fi', 'classic sci-fi', 'harrison ford', 'must see']


# 2. Word2Vec 모델 학습


## 1) 모델 학습
<br>
위처럼 생성한 배열을 Word2Vec 모델에 넣어 학습시킵니다. <br>
Word2Vec 모델은 Gensim의 라이브러리를 사용하였습니다. <br> 
<br>
Skip gram 방식을 사용하였고, Window Size는 256 차원입니다. <br>
100번 이하로 나온 태그들은 제거하고, 200번 학습을 시켰습니다. <br>


In [4]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(tag_array,
                 size=256,
                 window=3,
                 workers=8,
                 min_count=50,
                 iter=100,
                 sg=1)

model.init_sims(replace=True)


C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2) 모델 테스트

<br>

모델 학습이 잘 되었는지 테스트를 해봅니다. <br>
marvel 태그와 비슷한 태그를 출력합니다. <br>

In [5]:
print("spider-man 태그와 유사한 태그 목록 :  \n")
model.wv.most_similar("spider-man")


spider-man 태그와 유사한 태그 목록 :  



C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('thor', 0.5941746234893799),
 ('iron man', 0.5318883061408997),
 ('mcu', 0.5048591494560242),
 ('hulk', 0.4940865635871887),
 ('marvel', 0.4853953719139099),
 ('great villain', 0.4769175052642822),
 ('black widow', 0.4638150930404663),
 ('marvel cinematic universe', 0.4490388035774231),
 ('captain america', 0.44037163257598877),
 ('kat dennings', 0.43202510476112366)]

# 3. 태그 선별



## 1) 영화 추천에 사용할 유의미한 태그를 선별

<br>

영화 추천에 사용할 유의미한 태그를 선별합니다. <br>
과정이 매우 복잡하고 까다로운데, word2vec_mylib의 tag_manager.py 파일에 <br>
주석이 상세하게 달려있습니다. 직접 코드를 보며 이해하시는게 쉬울거라 생각됩니다.


In [22]:
from src.word2vec_mylib import tag_manager as tm
from src.word2vec_mylib import movie_manager as mm

movie = pd.read_csv('./data/movies_large.csv')
movie_df = pd.DataFrame(movie)

drop_tag_df = mm.get_movieId_tag_df(tag_df)

testMovieId = 1
TF_list = tm.get_semantic_tag(testMovieId, drop_tag_df)

recommend_tags = tm.get_recommend_tags(TF_list, model)

print()
print(mm.id_to_name(testMovieId, movie_df))
print()
print("영화 추천에 사용할 태그의 목록입니다 ", recommend_tags)
print()
# 추천에 사용할 태그를 선별하였습니다.



   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy

영화 추천에 사용할 태그의 목록입니다  ['oscar (best animated feature)', 'talking animals', 'animation', 'disney', 'pixar', 'disney animated feature', 'pixar animation']



C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):



# 4. 추천을 위한 전처리




## 1) 평점데이터 로드 

<br>

TF-IDF 모델을 적용하기위해 평점 데이터를 로드합니다. <br>
평점 데이터는 MovieLens의 데이터셋을 사용하였습니다. <br>



In [13]:
rating = pd.read_csv('./data/ratings_large.csv')
rating_df = pd.DataFrame(rating)
rating_df.head(10)

# 평점 데이터입니다.

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
5,1,1590,2.5,1256677236
6,1,1591,1.5,1256677475
7,1,2134,4.5,1256677464
8,1,2478,4.0,1256677239
9,1,2840,3.0,1256677500


## 2) 평균평점 3점 아래 영화 제거 
<br>

3점 아래의 영화는 추천해줄만한 영화가 아니라고 판단하고 <br>
추천 영화목록에서 제거합니다.<br>

In [15]:
from src.word2vec_mylib import movie_manager as mm

over_three_df = mm.get_over_three_movie(rating_df)
over_three_df.head(10)

# 평균평점 3점 이상의 영화 목록입니다.


,movieId
0,1
1,2
2,3
4,5
5,6
6,7
7,8
8,9
9,10
10,11



## 3) 태그를 가진 영화만 뽑아내기 
<br>

태그를 갖지 않은 영화는 태그를 비교하여 추천할 수 없기 떄문에, <br>
태그를 갖지 않은 영화를 제거합니다.

In [29]:
movieTag = tag_df[['movieId', 'tag']].sort_values(by=['movieId'])
over_three_list = []
for i in over_three_df.values:
    over_three_list.append(i[0])

semantic_df = tag_df[['movieId', 'tag']].sort_values(by=['movieId'])
semantic_df = semantic_df[semantic_df['movieId'].isin(over_three_list)]
semantic_df = semantic_df.drop_duplicates('movieId')
semantic_df = semantic_df.drop('tag', axis=1)
semantic_df  # <--- 추천에 사용할 영화 목록

semantic_list = []
for i in semantic_df.values:
    semantic_list.append(i[0])

semantic_df.head(10)

,movieId
708618,1
26414,2
907863,3
656610,5
938255,6
711494,7
543946,8
855573,9
720416,10
169441,11


# 5. 추천 섹션

## 1) 영화 데이터 로드
<br>
우리가 만든 모델이 잘 동작하는지 확인하기 위해 <br>
영화 데이터를 로드합니다. <br>
마찬가지로 MovieLens의 데이터를 사용하였습니다

In [26]:
movie = pd.read_csv('./data/movies_large.csv')
movie_df = pd.DataFrame(movie)
movie_df.head(10)

# 영화 이름과 장르 데이터입니다. 

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,u (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


## 2) 영화 데이터 로드
<br>
추천해줄수 있는 영화목록 (3점이상 + 태그보유)와 <br>
영화 데이터를 조인하여 추천해줄수 있는 영화 데이터 프레임을 만듭니다.

In [33]:
movie_df = movie_df[movie_df['movieId'].isin(semantic_list)]
movie_df.head(20)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,u (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller
10,11,"American President, The (1995)",Comedy|Drama|Romance


In [87]:
tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)

tag_df = tag_df[tag_df['movieId'].isin(semantic_list)]
tag_df = tag_df[tag_df.movieId != testMovieId]
tag_df = tag_df.sort_values(by=['movieId'])
tag_df = tag_df.drop('timestamp', axis=1)
tag_df = tag_df.drop('userId', axis=1)

tag_df

2 [['robin williams', 35], ['fantasy', 24], ['time travel', 20], ['board game', 13], ['animals', 12], ['bad cgi', 9], ['scary', 8], ['kid flick', 8], ['kirsten dunst', 5], ['magic board game', 5], ['children', 5], ['monkey', 4], ['based on a book', 3], ['adapted from:book', 3], ['childhood recaptured', 3], ['not for kids', 3], ['fiction', 2], ['family', 2], ['joe johnston', 2], ['chris van allsburg', 2], ['game', 2], ['herds of cgi animals', 2], ['time', 2], ['filmed in bc', 2], ['jungle', 2], ['adventure', 2], ['rainy day watchlist', 1], ['see also:zathura', 1], ['itaege', 1], ['lebbat', 1], ['recluse', 1], ['new home', 1], ['giant insect', 1], ['disappearance', 1], ["based on children's book", 1], ['for children', 1], ['dynamic cgi action', 1], ['saturn award (best supporting actress)', 1], ['saturn award (best special effects)', 1], ['kids', 1], ['horrifying (but not horror genre)', 1], ['comedy', 1], ['adaptation of book', 1], ['thrill', 1], ['childish', 1], ['clv', 1], ['baker viu

3 [['jack lemmon', 2], ['walter matthau', 2], ['moldy', 2], ['fishing', 2], ['sequel', 2], ['old', 2], ['old people that is actually funny', 1], ['ann margaret', 1], ['clv', 1], ['sophia loren', 1], ['comedinha de velhinhos engraã§ada', 1], ['grun running', 1], ['howard deutch', 1], ['burgess meredith', 1], ['daryl hannah', 1], ['good soundtrack', 1], ['old man', 1], ['no_fa_ganes', 1], ['funny', 1], ['sequel fever', 1], ['best friend', 1], ['duringcreditsstinger', 1], ['comedinha de velhinhos engraãƒâ§ada', 1], ['funniest movies', 1], ['comedy', 1]]



5 [['steve martin', 7], ['pregnancy', 4], ['family', 3], ['comedy', 2], ['wedding', 2], ['diane keaton', 2], ['touching', 2], ['childhood classics', 2], ['aging', 1], ['gynecologist', 1], ['sequel', 1], ['baby', 1], ['confidence', 1], ['contraception', 1], ['daughter', 1], ['midlife crisis', 1], ['parent child relationship', 1], ['worst movies ever', 1], ['remake', 1], ["it thought it was funny but it wasn't", 1], ['watched under duress', 1], ['sequel fever', 1], ['humorous', 1], ['clv', 1], ['fantasy', 1]]



KeyboardInterrupt: 

In [90]:
distinct_df = tag_df.drop_duplicates('movieId')
recommend_movies = distinct_df.movieId.values


numberOfAll = 0
minTagWeight = 0
temp = []

for tag, numberOfTimes in TF_list:
    numberOfAll += numberOfTimes
    # 태그 가중치의 총합을 구함

for tag, numberOfTimes in TF_list:
    weightForTag = (numberOfTimes / numberOfAll) * 10
    index = round(weightForTag)
    minTagWeight = index
    # 태그별 가중치를 구하고 모델을 사용할 최소가중치를 구함

for tag, numberOfTimes in TF_list:
    weightForTag = (numberOfTimes / numberOfAll) * 10
    index = round(weightForTag)
    if index != 0:  # 그영화의 모든 태그중 가중치가 10퍼센트 이상인 태그라면
        temp.append(tag)  # 그 영화의 태그를 리스트에 담음


recommend_TF_lists = []
for movieId in recommend_movies:
    print(movieId, tm.get_semantic_tag(movieId, tag_df))
    print()



2 [['robin williams', 35], ['fantasy', 24], ['time travel', 20], ['board game', 13], ['animals', 12], ['bad cgi', 9], ['scary', 8], ['kid flick', 8], ['kirsten dunst', 5], ['magic board game', 5], ['children', 5], ['monkey', 4], ['based on a book', 3], ['adapted from:book', 3], ['childhood recaptured', 3], ['not for kids', 3], ['fiction', 2], ['family', 2], ['joe johnston', 2], ['chris van allsburg', 2], ['game', 2], ['herds of cgi animals', 2], ['time', 2], ['filmed in bc', 2], ['jungle', 2], ['adventure', 2], ['rainy day watchlist', 1], ['see also:zathura', 1], ['itaege', 1], ['lebbat', 1], ['recluse', 1], ['new home', 1], ['giant insect', 1], ['disappearance', 1], ["based on children's book", 1], ['for children', 1], ['dynamic cgi action', 1], ['saturn award (best supporting actress)', 1], ['saturn award (best special effects)', 1], ['kids', 1], ['horrifying (but not horror genre)', 1], ['comedy', 1], ['adaptation of book', 1], ['thrill', 1], ['childish', 1], ['clv', 1], ['baker viu

3 [['jack lemmon', 2], ['walter matthau', 2], ['moldy', 2], ['fishing', 2], ['sequel', 2], ['old', 2], ['old people that is actually funny', 1], ['ann margaret', 1], ['clv', 1], ['sophia loren', 1], ['comedinha de velhinhos engraã§ada', 1], ['grun running', 1], ['howard deutch', 1], ['burgess meredith', 1], ['daryl hannah', 1], ['good soundtrack', 1], ['old man', 1], ['no_fa_ganes', 1], ['funny', 1], ['sequel fever', 1], ['best friend', 1], ['duringcreditsstinger', 1], ['comedinha de velhinhos engraãƒâ§ada', 1], ['funniest movies', 1], ['comedy', 1]]



5 [['steve martin', 7], ['pregnancy', 4], ['family', 3], ['comedy', 2], ['wedding', 2], ['diane keaton', 2], ['touching', 2], ['childhood classics', 2], ['aging', 1], ['gynecologist', 1], ['sequel', 1], ['baby', 1], ['confidence', 1], ['contraception', 1], ['daughter', 1], ['midlife crisis', 1], ['parent child relationship', 1], ['worst movies ever', 1], ['remake', 1], ["it thought it was funny but it wasn't", 1], ['watched under duress', 1], ['sequel fever', 1], ['humorous', 1], ['clv', 1], ['fantasy', 1]]



KeyboardInterrupt: 